In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy.io

from keras.models import Sequential
from keras.models import model_from_json
from keras.layers import Dense, Conv2D, Flatten
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model

Using TensorFlow backend.


# Deep Learning study on the results of the 1D Pseudo-Wigner Distribution using Neural Networks

**Why?**

Check if the wigner distribution of an hologram is capable to predict how many point sources generated the hologram (1 to 5 sources).

**How?**

Using a convolutional neural networks to solve this classification problem.

**What?**

Using the keras libray (python).

**Some examples:**
* https://towardsdatascience.com/building-a-convolutional-neural-network-cnn-in-keras-329fbbadc5f5 

**Load dataset**

In [2]:
path = 'output/wigner_distribution/'
file_name = 'wd_results.npy'

dataset = np.load(path + file_name)
print(dataset.shape)

(125, 8, 200, 200)


## CNN (Convolutional Neural Networks)

**Data pre-processing**

In [3]:
def compute_targets_array(nb_class, X_train):
    """
    Compute an array with the targets of the dataset. Note that the number on the array correspond to the number of 
    sources minus one
    """
    # Number of the examples
    nb_holograms = X_train.shape[0]
    
    # Number of examples per class
    nb_holograms_class = int(nb_holograms / nb_class)
    
    # Y vector
    Y_array = np.zeros((nb_holograms,))
    counter = 1
    target = 0
    
    for i in range(nb_holograms):
        if counter == (nb_holograms_class + 1):
            target = target + 1
            counter = 1
        Y_array[i,] = target
        counter = counter + 1    
    
    return Y_array

In [5]:
# Select one of the 8 frequencies ! BUG, MUST FIX
X_train = dataset[:,0,:,:]

# The 1 signify that the images are greyscale
X_train = X_train.reshape(125,200,200,1)

print(X_train.shape)

(125, 200, 200, 1)


In [6]:
# Compute array of targets
nb_class = 5
Y_array = compute_targets_array(nb_class, X_train)

print(Y_array.shape)

# One-hot encode target column
Y_train = to_categorical(Y_array)
print(Y_train.shape)

(125,)
(125, 5)


**Building the model**

In [7]:
# Create model
model = Sequential() # allows build a model layer by layer

# Add model layers

# Conv2D layer: 
# 64 nodes, 3x3 filter matrix, Rectified Linear Activation as activation function,
# shape of each input (200, 200, 1,) with 1 signifying images are greyscale
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(200,200,1))) 

# 32 nodes
model.add(Conv2D(32, kernel_size=3, activation='relu'))

# Flatten layer: connection between the convolution and dense layers
model.add(Flatten())

# Dense layer: used for the output layer
# 5 nodes for the output layer, one for each possible outcome (1-5)
# 'softmax' as activation function, it makes the output sump up to 1 so the output
# can be interpreted as probalities
model.add(Dense(5, activation='softmax'))

**Compiling the model**

In [8]:
# Three parameters:
# optmizer: 'adam'
# loss function: 'categorical_crossentropy', the most common choice for classification
# metrics: 'accuracy', to see the accuracy score
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

**Training the model**

In [9]:
# Number of epochs: number of tmes the model wil cycle trough the data
model.fit(X_train, Y_train, validation_data=(X_train, Y_train), epochs=2)

Train on 125 samples, validate on 125 samples
Epoch 1/2
125/125 [==============================] - 27s 212ms/step - loss: 1566685.0600 - accuracy: 0.1760 - val_loss: 2101887.7750 - val_accuracy: 0.2000
Epoch 2/2
125/125 [==============================] - 25s 201ms/step - loss: 1084067.4833 - accuracy: 0.2000 - val_loss: 126335.1262 - val_accuracy: 0.2160


**Evalutation**

In [10]:
# Evaluate the keras model
_, accuracy = model.evaluate(X_train, Y_train, verbose=0)
print('Accuracy: %.2f%%' % (accuracy*100))

Accuracy: 21.60%


**Make predictions**

In [11]:
# Make probability predictions with the model
predictions = model.predict(X_train)

# Round predictions 
rounded = [round(x[0]) for x in predictions]

# Make class predictions with the model
predictions = model.predict_classes(X_train)

# Summarize the first 5 cases
for i in range(5):
    print('Predicted: %d (expected: %d)' % (predictions[i], Y_array[i]))

Predicted: 4 (expected: 0)
Predicted: 4 (expected: 0)
Predicted: 4 (expected: 0)
Predicted: 4 (expected: 0)
Predicted: 4 (expected: 0)


**Save weights and model**

In [12]:
# Serialize model to JSON
model_json = model.to_json()
with open("output/neural_networks/model.json", "w") as json_file:
    json_file.write(model_json)
    
# Serialize weights to HDF5
model.save_weights("output/neural_networks/model.h5")
print("Saved model structure and weights")

Saved model structure and weights


**Load model**

In [15]:
# The model weights and architecture were saved separated, so it must re-compile

# Load json and create model
json_file = open('output/neural_networks/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# Load weights into new model
loaded_model.load_weights("output/neural_networks/model.h5")
print("Loaded model from disk")
 
# Evaluate loaded model on test data
loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
score = loaded_model.evaluate(X_train, Y_train, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Loaded model from disk
accuracy: 21.60%


**Summary**

In [16]:
# Summarize model.
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 198, 198, 64)      640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 196, 196, 32)      18464     
_________________________________________________________________
flatten_1 (Flatten)          (None, 1229312)           0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 6146565   
Total params: 6,165,669
Trainable params: 6,165,669
Non-trainable params: 0
_________________________________________________________________


**Plot model**

In [17]:
# Error, BUG, MUST FIX

# plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)